In [15]:
from urllib.parse import urlparse
from urllib.parse import parse_qs
import math
from youtube_transcript_api import YouTubeTranscriptApi

In [16]:
yt_video_list=["https://www.youtube.com/watch?v=4qh_rguqdD4",
               "https://www.youtube.com/watch?v=rR_mlBb-5-k&t=1211s"
               ,"https://www.youtube.com/watch?v=k-4kdiEqCYg",
               "https://www.youtube.com/watch?v=GX4nd41Iu-Q",
               "https://www.youtube.com/watch?v=L65CxuCt-bg",
               "https://www.youtube.com/watch?v=hSXlv26f_Kg&t=783s",
               "https://www.youtube.com/watch?v=BYkPDg7RZa4",
               "https://www.youtube.com/watch?v=_2k1pHKSHtg",
               "https://www.youtube.com/watch?v=ibQn5wau_VA",
               "https://www.youtube.com/watch?v=Jie6URfi1HY"]

In [17]:
list_of_chunks_all_videos=[]    
for video_url in yt_video_list:
    list_of_lines=[]  #for current video
    url_data=urlparse(video_url)
    query = parse_qs(url_data.query)
    video_id = query["v"][0]
    print(video_id)
    transcritps = YouTubeTranscriptApi.get_transcript(video_id)
    for i in transcritps:
        time=i['start']
        time=math.floor(time)
        hours=int(time//3600)
        minutes=int((time%3600)//60)
        secs=time%60
        # print( f"{hours:02}:{minutes:02}:{secs:06.3f}")
        i['start']=f"{hours:02}:{minutes:02}:{secs:06.3f}"
        ## and adding youtube video
        i['video_url']=video_url   # adding youtube video url
        list_of_lines.append({'time':i['start'],'text':i['text'],'video_url':i['video_url']})
    list_of_chunks=[]
    chunk=[]
    chunk.append(list_of_lines[0])
    chunk.append(list_of_lines[1])
    no=0
    for i in list_of_lines[2:]:
        chunk.append(i)
        no+=1
        if len(chunk)>8:
            chunk.pop(0)
        if (no)%6==0:
            chunk_start=chunk[0]['time']
            chunk_url=chunk[0]['video_url']
            chunk_text=''
            count=0
            for j in chunk:
                count+=1
                chunk_text+=j['text']
                chunk_text+=' '
            # print(count)
            # print(chunk_text)
            # print()
            list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})
            no=0
    if len(chunk)!=0:
        chunk_start=chunk[0]['time']
        chunk_url=chunk[0]['video_url']
        chunk_text=''
        count=0
        for j in chunk:
            count+=1
            chunk_text+=j['text']
            chunk_text+=' '
        # print(count)
        # print(chunk_text)
        # print()
        list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})
    no=0
    print(len(list_of_chunks))
    list_of_chunks_all_videos.extend(list_of_chunks)
    print("the len now is ",len(list_of_chunks_all_videos))
    

4qh_rguqdD4
101
the len now is  101
rR_mlBb-5-k
99
the len now is  200
k-4kdiEqCYg
100
the len now is  300
GX4nd41Iu-Q
104
the len now is  404
L65CxuCt-bg
117
the len now is  521
hSXlv26f_Kg
243
the len now is  764
BYkPDg7RZa4
117
the len now is  881
_2k1pHKSHtg
107
the len now is  988
ibQn5wau_VA
17
the len now is  1005
Jie6URfi1HY
91
the len now is  1096


In [18]:
print(list_of_chunks_all_videos)

[{'time': '00:00:00.000', 'text': "e [Music] [Music] hello hello and happy Monday it's a May bank holiday at least in the UK I hope it you're having a wonderful Monday either way wherever you are if you're not having a bank holiday my name is ", 'video_url': 'https://www.youtube.com/watch?v=4qh_rguqdD4'}, {'time': '00:00:50.000', 'text': "either way wherever you are if you're not having a bank holiday my name is Mandy Seline I'm a director and DP and I'm the founder of breadbox Studios which is a boutique post and production house kind of based in the UK we specialize in branded and digital first content in general so we work with a lot ", 'video_url': 'https://www.youtube.com/watch?v=4qh_rguqdD4'}, {'time': '00:01:01.000', 'text': "specialize in branded and digital first content in general so we work with a lot of creators as well for those of you who don't know the last few Mondays of May I've been hosting these challenges every Monday and it's the last Monday of May so this is my la

In [19]:
# put into a df and vectorizing

In [20]:
import pandas as pd

In [21]:
df=pd.DataFrame(list_of_chunks_all_videos)

In [22]:
df.shape

(1096, 3)

In [23]:
# df.head(400)

In [24]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="InsecureRequestWarning")

In [25]:
from opensearchpy import OpenSearch
from opensearch_dsl import Search

In [26]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.
ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)

In [27]:
from opensearchpy import OpenSearch

CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client

client = get_os_client()

/Users/vpatwal/Desktop/trying_summarization/.venv/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [28]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "all-MiniLM-L6-v2"


In [29]:
model = SentenceTransformer(MODEL_NAME)

In [30]:
EMBEDDING_DIM = model.encode(["Sample sentence"])[0].shape[0]
print(EMBEDDING_DIM)#will be used ahead

384


In [31]:
index_name = "10_chunks"

index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": { #how do we store, 
    "properties": {
        "embedding": {
          "type": "knn_vector", #we are going to put 
          "dimension": EMBEDDING_DIM,
          "method": {
            "name": "hnsw",
            "space_type": "l2",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
         }
     }
    }
}}


In [32]:
response = client.indices.delete(index = index_name)

In [33]:
response = client.indices.create(index=index_name, body=index_body)

In [34]:
df.head()
df.columns

Index(['time', 'text', 'video_url'], dtype='object')

In [35]:
import pandas as pd

for index, row in df.iterrows():
    print(f"Id: {row['time']}, Start: {row['time']}, Overview: {row['text']},url:{row['video_url']}")
    # original_title = row['time']
    overview = row['text']
    id = row['time']
    st_time=row['time']
    url=row['video_url']
    # Sentence transformer model takes list of documents as input and returns list of embeddings.
    embedding = model.encode([overview])[0]
    # We are inserting a data point with 3 attribute, "id", "text" and "embedding" as knn_vector type.
    my_doc = {"id": id, "plot": overview, "embedding": embedding,"time" : st_time,"url":url}
    res = client.index(
        index=index_name,
        body=my_doc,
        id = str(index),
        refresh = True
        )

Id: 00:00:00.000, Start: 00:00:00.000, Overview: e [Music] [Music] hello hello and happy Monday it's a May bank holiday at least in the UK I hope it you're having a wonderful Monday either way wherever you are if you're not having a bank holiday my name is ,url:https://www.youtube.com/watch?v=4qh_rguqdD4
Id: 00:00:50.000, Start: 00:00:50.000, Overview: either way wherever you are if you're not having a bank holiday my name is Mandy Seline I'm a director and DP and I'm the founder of breadbox Studios which is a boutique post and production house kind of based in the UK we specialize in branded and digital first content in general so we work with a lot ,url:https://www.youtube.com/watch?v=4qh_rguqdD4
Id: 00:01:01.000, Start: 00:01:01.000, Overview: specialize in branded and digital first content in general so we work with a lot of creators as well for those of you who don't know the last few Mondays of May I've been hosting these challenges every Monday and it's the last Monday of May so

In [52]:
""" Example query text """
# user_query = "Applying Lens Blur"
# user_query = "hide content"
# user_query = "expanding"
# user_query="deleting people"
# user_query='How does the Generative AI Remove feature in Lightroom work?'
# user_query="What steps does Jacob describe for creating a drop shadow in Illustrator?"
# user_query="Inserting drop shadow"
# user_query="cut it"
# user_query="How to create a montage using markers in Premiere Pro?"
# user_query="select and arrange photos for a video montage?"
# user_query="add markers to a timeline for precise editing"
# user_query="delete unwanted sections of a video sequence in Premiere Pro"
user_query="Asset Availability and Customization"


""" Embedding the query by using the same model """
query_embedding = model.encode((user_query))


query_body = {
    "query": {"knn": {"embedding": {"vector": query_embedding, "k": 10}}},
    "_source": False,
    "fields": ["id", "title", "plot","time","url"],
    "size":10  # vivek added this , top 10
}

results = client.search(
    body=query_body,
    index=index_name
)

In [37]:
for i, result in enumerate(results["hits"]["hits"]):
    # print(i,results)
    plot = result['fields']['plot'][0]
    url = result['fields']['url'][0]
    score = result['_score']
    time=result['fields']['time'][0]
    print(f"{i+1}., Start Time: {time} Score: {score}, Url: {url}  Plot: {plot}")

1., Start Time: 00:18:28.000 Score: 0.51300114, Url: https://www.youtube.com/watch?v=hSXlv26f_Kg&t=783s  Plot: along inside your asset what does this flexibility look like in the real world scenario it's important to know the status of all the work that's coming together but our out of the-box status doesn't really map to the way you think about your team's post- production you can now take the same status field you 
2., Start Time: 00:01:15.000 Score: 0.5096753, Url: https://www.youtube.com/watch?v=k-4kdiEqCYg  Plot: around I am super sure today you will see assets used that may be different inside your pack but you can also use your own assets so please do not feel tied to what I'm showing you I will be using 50 photos you can use whatever you like like um I'll tell you a bit more about what we're going to be doing today 
3., Start Time: 00:01:44.000 Score: 0.49734426, Url: https://www.youtube.com/watch?v=4qh_rguqdD4  Plot: we're just going to demystify exporting a little bit because

In [45]:
# s = Search(using=client, index=index_name) \
#     .filter("term", url="https://www.youtube.com/watch?v=hSXlv26f_Kg&t=783s") \
#     .query("match", title="image")

response = s.execute()

In [46]:
print(response)

<Response: {}>


In [47]:
type(response)

opensearch_dsl.response.Response

In [53]:
import requests

In [55]:
import requests
 
# defining the api-endpoint
url = "https://localhost:9200"
 
# # your API key here
# API_KEY = "XXXXXXXXXXXXXXXXX"
 
r = requests.get(url = url)

SSLError: HTTPSConnectionPool(host='localhost', port=9200): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))